## Importing libraries

In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import pandas as pd
from IPython.display import display, HTML

Using TensorFlow backend.


## Import Training data(Acled) into dataframes

In [185]:
col=['Text']
riots_pos = pd.read_csv('/Users/harsha/Documents/cse635_AIR/Project/Main/Data/Riots_pos.tsv', delimiter ='\t', quoting = 3,usecols=col)
violence_pos = pd.read_csv('/Users/harsha/Documents/cse635_AIR/Project/Main/Data/Violence_pos.tsv', delimiter='\t', quoting= 3,usecols=col)
riots_neg = pd.read_csv('/Users/harsha/Documents/cse635_AIR/Project/Main/Data/Riots_neg.tsv', delimiter ='\t', quoting = 3,usecols=col)
violence_neg = pd.read_csv('/Users/harsha/Documents/cse635_AIR/Project/Main/Data/Violence_neg.tsv', delimiter='\t', quoting= 3,usecols=col)
negative = pd.read_csv('/Users/harsha/Documents/cse635_AIR/Project/Main/Data/Negative.tsv', delimiter='\t', quoting= 3,usecols=col)
negative_other = pd.read_csv('/Users/harsha/Documents/cse635_AIR/Project/Main/Data/Negative_other.tsv', delimiter='\t', quoting= 3)

In [186]:
riots_pos = riots_pos.sample(n=2000).reset_index(drop=True)
violence_pos= violence_pos.sample(n=2000).reset_index(drop=True)
negative = negative.sample(n=2000).reset_index(drop=True)
# negative_other = negative_other.sample(n=500).reset_index(drop=True)

In [147]:
# negative = pd.concat([negative,negative_other],ignore_index=True).sample(n=2000).reset_index(drop=True)

In [187]:
Category =[]
for i in range(0,6000):
    if i<2000:
        Category.append(1)
    elif i<4000:
        Category.append(2)
    else:
        Category.append(0)

In [188]:
dataset = pd.concat([riots_pos,violence_pos],ignore_index=True)
dataset = pd.concat([dataset,negative],ignore_index=True)

In [189]:
dataset['Category'] = Category[:]

In [169]:
display(HTML(dataset.to_html()))

,Text,Category
0,"""On April 2, members of Dalit's community demo...",1
1,Clashes erupted between youth rioters and secu...,1
2,Nurses and paramedical staff from Rajindra hos...,1
3,"""On August 28, locals blocked roads by burning...",1
4,"""On 15 Feb, unidentified mobs targeted Kashmir...",1
5,"""A day after IIT students representing the Amb...",1
6,"""Angered by the 'Auill-treatment'Au meted out ...",1
7,"""A joint forum constituted to demand justice f...",1
8,"""On 19 October, rioting groups of rice millers...",1
9,"""On 31 January, Scheduled Caste students stage...",1


In [190]:
dataset = dataset.sample(frac=1).reset_index(drop=True)

## Cleaning the dataset and generate Corpus

In [191]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [192]:
def cleanAndGenCorpus(dataset):
    corpus =[]
    for i in range(0,dataset['Text'].size):
        text= re.sub('[^a-zA-z&]',' ',dataset['Text'][i])
        text= text.lower()
        text= text.split()
        ps= PorterStemmer();
        text= [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
        text= ' '.join(text)
        corpus.append(text)
    return corpus

In [193]:
corpus = cleanAndGenCorpus(dataset)

## Feature Extraction

In [194]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [216]:
tf= TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',max_features=5000)
X= tf.fit_transform(corpus).toarray()
Y= dataset.iloc[:,1].values

In [217]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

In [218]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=0)
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## Training the classifier

In [219]:
classifier= GaussianNB()
classifier.fit(X_train,Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [220]:
Y_pred = classifier.predict(X_test)

In [221]:
cm = confusion_matrix(Y_test,Y_pred)

In [222]:
display(cm)

array([[245, 109, 147],
       [ 17, 413,  51],
       [ 55, 191, 272]])

In [223]:
from sklearn.externals import joblib 
joblib.dump(tf,'tf')
joblib.dump(classifier,'classifier.pkl')

['classifier.pkl']

In [50]:
corpus_violence = cleanAndGenCorpus(violence_pos)
vec = CountVectorizer(max_features=100).fit(corpus_violence)
bag_of_words = vec.transform(corpus_violence)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [51]:
print(words_freq)

[('attack', 700), ('polic', 562), ('shot', 423), ('kill', 407), ('injur', 399), ('unidentifi', 381), ('group', 312), ('worker', 264), ('assault', 263), ('leader', 252), ('fire', 246), ('suspect', 240), ('dead', 230), ('allegedli', 228), ('milit', 216), ('local', 208), ('youth', 201), ('arm', 201), ('hous', 189), ('injuri', 179), ('peopl', 176), ('activist', 167), ('disput', 162), ('station', 162), ('hospit', 158), ('victim', 147), ('support', 145), ('famili', 144), ('person', 136), ('beaten', 135), ('elect', 134), ('assail', 132), ('home', 126), ('tehsil', 125), ('unknown', 122), ('beat', 121), ('abduct', 120), ('death', 117), ('incid', 117), ('reason', 115), ('dalit', 114), ('bengal', 114), ('former', 107), ('resid', 106), ('land', 105), ('offic', 104), ('bihar', 104), ('accus', 103), ('panchayat', 100), ('report', 100), ('maoist', 94), ('inform', 92), ('student', 91), ('also', 88), ('alleg', 85), ('violenc', 84), ('murder', 84), ('villag', 81), ('bodi', 80), ('found', 79), ('candid',

In [97]:
corpus_riots = cleanAndGenCorpus(riots_pos)
vec = CountVectorizer(max_features=100).fit(corpus_riots)
bag_of_words = vec.transform(corpus_riots)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
print(words_freq)

[('protest', 1583), ('stage', 859), ('demand', 536), ('govern', 330), ('demonstr', 295), ('student', 295), ('offic', 218), ('held', 207), ('union', 191), ('road', 178), ('activist', 174), ('clash', 166), ('resid', 165), ('farmer', 155), ('block', 154), ('local', 152), ('report', 152), ('peopl', 148), ('support', 137), ('nation', 135), ('arrest', 135), ('employe', 133), ('size', 130), ('minist', 126), ('ralli', 122), ('front', 119), ('outsid', 118), ('part', 116), ('includ', 112), ('group', 108), ('attack', 105), ('teacher', 105), ('dharna', 104), ('youth', 104), ('alleg', 102), ('leader', 100), ('school', 98), ('call', 97), ('injur', 97), ('kill', 95), ('univers', 95), ('organ', 94), ('commun', 87), ('burnt', 87), ('bathinda', 86), ('committe', 85), ('associ', 85), ('effigi', 84), ('strike', 82), ('bandh', 82), ('station', 82), ('tn', 81), ('highway', 77), ('colleg', 74), ('new', 74), ('organis', 72), ('led', 72), ('stone', 71), ('chief', 71), ('bill', 71), ('hundr', 71), ('agit', 68),

In [224]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(cm, index = ["None", "Riot/Protest","Violence"],
                  columns = ["Predicted None ", "Predicted Riot/Protest","Predicted Violence"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,fmt='g')